In [1]:
!pip install requests beautifulsoup4 -q

In [15]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
from selenium.webdriver.common.by import By
import time
from datetime import datetime
from datetime import date

In [16]:
def get_data(left_line, middle_line):
    # 데이터를 저장할 리스트
    data_list = []
    # 날짜 값 임시 할당
    temp_date = date.today()
    city_do = ''
    for idx in range(2, 273): # 3809
        data_dict = {
            'upload_time': date.today()
            , 'city_do' : ''
            , 'city_gun_gu' : ''
            , '승용' : 0
            , '승합' : 0
            , '화물' : 0
            , '특수' : 0
            , 'total_count' : 0
        }
        # 지역 라인( 왼쪽 ) 값 가져오기
        left_line = left_tbody[0].find('tr', attrs={"onmousemove": f'Grids[0].ARow=Grids[0].Rows["{idx}"];Grids[0].ASec=0;'})
        # 수치 값( 오른쪽 ) 가져오기
        middle_line = middle_tbody[0].find('tr', attrs={"onmousemove": f'Grids[0].ARow=Grids[0].Rows["{idx}"];Grids[0].ASec=0;'})
    
        # 왼쪽 라인 내부 요소 가져오기
        left_word = left_line.find_all('td')
        # 오른쪽 라인 내부 요소 가져오기
        middle_word = middle_line.find_all('td')
    
        # 요소 개수 구하기
        td_len = len(left_word)
    
        # 날짜 바뀜
        if td_len == 4: 
            temp_date = datetime.strptime(left_word[1].text, "%Y-%m").date()
            # print(temp_date)
            city_do = left_word[2].text
            data_dict['city_gun_gu'] = left_word[3].text
        
         # 시/도 바뀜
        elif td_len == 3:
            city_do = left_word[1].text
            data_dict['city_gun_gu'] = left_word[2].text
        
        # 시/군/구 바뀜
        elif td_len == 2: 
            data_dict['city_gun_gu'] = left_word[1].text
    
        # 날짜, 시/도 적용
        data_dict['upload_time'] = temp_date
        data_dict['city_do'] = city_do
    
        # 수치 적용
        data_dict['승용'] = middle_line.find('td', class_='GMClassReadOnly GMWrap0 GMFloat GMCell IBSheetFont0 HideCol0C7').text
        data_dict['승합'] = middle_line.find('td', class_='GMClassReadOnly GMWrap0 GMFloat GMCell IBSheetFont0 HideCol0C11').text
        data_dict['화물'] = middle_line.find('td', class_='GMClassReadOnly GMWrap0 GMFloat GMCell IBSheetFont0 HideCol0C15').text
        data_dict['특수'] = middle_line.find('td', class_='GMClassReadOnly GMWrap0 GMFloat GMCell IBSheetFont0 HideCol0C19').text
        data_dict['total_count'] = middle_line.find('td', class_='GMClassReadOnly GMWrap0 GMFloat GMCell IBSheetFont0 HideCol0C23').text
    
        # 적용된 딕셔너리 데이터 리스트에 추가
        data_list.append(data_dict)
    return data_list

In [29]:
driver = webdriver.Chrome()

# 시작 날짜
start_year = 24
start_month = 1

# 끝 날짜
end_year = 25
end_month = 2

# 현재 날짜
current_year = start_year
current_month = start_month

# 결과 데이터 리스트
result_list = []

# 모든 날짜를 순회할 동안 반복
while True:
    # 현재 날짜에 따른 url 설
    url = f"https://stat.molit.go.kr/portal/cate/statView.do?hRsId=58&hFormId=5498&hSelectId=5498&hPoint=00&hAppr=1&hDivEng=&oFileName=&rFileName=&midpath=&sFormId=5498&sStart=20{current_year}{current_month:02}&sEnd=20{current_year}{current_month:02}&sStyleNum=2&settingRadio=xlsx"
    print(url)
    driver.get(url)

    time.sleep(1)

    # BeautifulSoup 객체 생성
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 지역 섹션과 값 섹션 추출
    left_tbody = soup.select('div.GMBodyLeft > div.GMPageOne > table > tbody')
    middle_tbody = soup.select('div.GMBodyMid > div.GMPageOne > table > tbody')

    # 크롤링 후 결과 데이터 리스트에 추가
    result_list += get_data(left_tbody, middle_tbody)

    # 크롤링 후 다음 날짜로 넘기기
    current_month += 1
    if current_month >= 12:
        current_month = 1
        current_year += 1

    # 현재 날짜가 끝 날짜를 넘었을 때 종료
    if current_year == end_year and current_month > end_month:
        break

    
time.sleep(1)
# 브라우저 종료
driver.quit()

https://stat.molit.go.kr/portal/cate/statView.do?hRsId=58&hFormId=5498&hSelectId=5498&hPoint=00&hAppr=1&hDivEng=&oFileName=&rFileName=&midpath=&sFormId=5498&sStart=202401&sEnd=202401&sStyleNum=2&settingRadio=xlsx
https://stat.molit.go.kr/portal/cate/statView.do?hRsId=58&hFormId=5498&hSelectId=5498&hPoint=00&hAppr=1&hDivEng=&oFileName=&rFileName=&midpath=&sFormId=5498&sStart=202402&sEnd=202402&sStyleNum=2&settingRadio=xlsx
https://stat.molit.go.kr/portal/cate/statView.do?hRsId=58&hFormId=5498&hSelectId=5498&hPoint=00&hAppr=1&hDivEng=&oFileName=&rFileName=&midpath=&sFormId=5498&sStart=202403&sEnd=202403&sStyleNum=2&settingRadio=xlsx
https://stat.molit.go.kr/portal/cate/statView.do?hRsId=58&hFormId=5498&hSelectId=5498&hPoint=00&hAppr=1&hDivEng=&oFileName=&rFileName=&midpath=&sFormId=5498&sStart=202404&sEnd=202404&sStyleNum=2&settingRadio=xlsx
https://stat.molit.go.kr/portal/cate/statView.do?hRsId=58&hFormId=5498&hSelectId=5498&hPoint=00&hAppr=1&hDivEng=&oFileName=&rFileName=&midpath=&sFor

In [ ]:
# 결과 확인용
result_list

# pandas로 csv 변환

In [21]:
import pandas as pd 

In [30]:
df = pd.DataFrame(result_list)
df.head(5)

,upload_time,city_do,city_gun_gu,승용,승합,화물,특수,total_count
0,2024-01-01,서울,강남구,"233,362","4,624","16,281",536,"254,803"
1,2024-01-01,서울,강동구,"134,641","4,198","14,795",444,"154,078"
2,2024-01-01,서울,강북구,"62,730","3,006","8,466",275,"74,477"
3,2024-01-01,서울,강서구,"178,148","5,714","20,983",684,"205,529"
4,2024-01-01,서울,관악구,"102,805","3,516","11,581",376,"118,278"


In [ ]:
df.to_csv('../data/vehicle_data.csv', encoding='cp949', index = False)

In [32]:
df

,upload_time,city_do,city_gun_gu,승용,승합,화물,특수,total_count
0,2024-01-01,서울,강남구,"233,362","4,624","16,281",536,"254,803"
1,2024-01-01,서울,강동구,"134,641","4,198","14,795",444,"154,078"
2,2024-01-01,서울,강북구,"62,730","3,006","8,466",275,"74,477"
3,2024-01-01,서울,강서구,"178,148","5,714","20,983",684,"205,529"
4,2024-01-01,서울,관악구,"102,805","3,516","11,581",376,"118,278"
...,...,...,...,...,...,...,...,...
3518,2025-02-01,전북,장수군,"7,985",407,"4,623",73,"13,088"
3519,2025-02-01,전북,전주시 덕진구,"155,133","4,423","21,875",969,"182,400"
3520,2025-02-01,전북,전주시 완산구,"139,622","3,738","17,635",638,"161,633"
3521,2025-02-01,전북,정읍시,"43,065","1,836","16,043",385,"61,329"
